In [28]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from PIL import Image
import numpy as np
import cv2
from pytesseract import Output
import pandas as pd
from pdfminer.high_level import extract_text
import re

In [2]:
# Loading Labeled Dataset
# x_train, y_train = load_labeled_data()

In [29]:
img_path = r"c:/Users/edominer/Python Project/Extracting Text from Invoice/Edominer-1201_page-1.jpg"
annotations_path = r"c:/Users/edominer/Python Project/Extracting Text from Invoice/labeled data/Edominer-1201_page-1.xml"

In [31]:
def load_image(img_path):
    img = cv2.imread(img_path)
    
    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not load image at path: {img_path}")
        return FileNotFoundError  # raise an exception 
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

image = load_image(img_path)

# Check if image loading was successful before proceeding
if image is not None:
    print("Image Shape:",image.shape)
else:
    print("Image loading failed. Exiting.")

Image Shape: (3505, 2480, 3)


In [4]:
print("Image Shape:",image.shape)

Image Shape: (3505, 2480, 3)


In [32]:
import os
from lxml import etree

def load_annotations(xml_path):
    tree = etree.parse(xml_path)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bboxes.append([xmin, ymin, xmax, ymax])
    return bboxes



bboxes = load_annotations(annotations_path)

In [33]:
print("Bounding Boxes:",bboxes)

Bounding Boxes: [[110, 368, 1391, 462], [110, 460, 902, 523], [117, 523, 900, 579], [419, 643, 1089, 702], [421, 700, 1085, 847], [421, 843, 1078, 896], [1104, 647, 1536, 738], [1564, 647, 1987, 738], [419, 904, 1085, 947], [419, 953, 1087, 998], [413, 1002, 1078, 1053], [413, 1055, 1076, 1104], [151, 1174, 1085, 1236], [151, 1234, 1087, 1377], [153, 1385, 1078, 1430], [153, 1438, 698, 1479], [717, 1436, 1074, 1485], [151, 1483, 708, 1534], [153, 1700, 2023, 1751], [172, 1749, 1502, 3206]]


In [34]:
#training data
x_train = np.array([image])
y_train = np.array(bboxes)

print(x_train.shape)
print(y_train.shape)

(1, 3505, 2480, 3)
(20, 4)


In [8]:
# model =Sequential()
# model.add(Embedding(10000,128))
# model.add(LSTM(128))
# model.add(Dense(20,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam',matrices=['accuracy'])
# model.fit(x_train, y_train, epochs=5)

# print("Model training complete.")


TypeError: Trainer.compile() got an unexpected keyword argument 'matrices'

In [35]:
import numpy as np

# If you have a single image, expand its dimension to match batch size
images = np.expand_dims(image, axis=0)  # Now the shape is (1, 3505, 2480, 3)


In [38]:
import tensorflow as tf

# Annotations is a NumPy array of shape (20, 4)
annotations = np.array(bboxes)

# Create a dataset from images and annotations
dataset = tf.data.Dataset.from_tensor_slices((images.repeat(len(annotations), axis=0), annotations))

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=20).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


MemoryError: Unable to allocate 497. MiB for an array with shape (20, 3505, 2480, 3) and data type uint8

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(3505, 2480, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    
    # Output for 4 coordinates of bounding box
    tf.keras.layers.Dense(4)  
])

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.fit(dataset, epochs=10)
print("Model training complete.")


c:\Users\edominer\Python Project\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[69423648,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2] name: 

# Solution of Out of Memory Error

In [27]:
def resize_image(image, target_size=(256, 256)):
    return cv2.resize(image, target_size)

# image_path = 'path/to/your/image.jpg'
image = cv2.imread(img_path)
image_resized = resize_image(image)
image_resized.shape

(256, 256, 3)

In [ ]:
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image)
    image = image / 255.0
    return 

In [25]:
def resize_image(image, target_size=(256, 256)):
    return cv2.resize(image, target_size)

image_resized = resize_image(image)
image_resized.shape

(256, 256, 3)

In [26]:

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image)
    image = image / 255.0
    return image

def load_and_preprocess_from_path_label(path, xmin, ymin, xmax, ymax, label):
    return load_and_preprocess_image(path), (xmin, ymin, xmax, ymax, label)

paths = ['path/to/image1.jpg', 'path/to/image2.jpg']  # Example paths
xmins = [100, 200]
ymins = [100, 200]
xmaxs = [150, 250]
ymaxs = [150, 250]
labels = [1, 0]

dataset = tf.data.Dataset.from_tensor_slices((paths, xmins, ymins, xmaxs, ymaxs, labels))
dataset = dataset.map(load_and_preprocess_from_path_label)
dataset = dataset.shuffle(buffer_size=1000).batch(4).prefetch(tf.data.experimental.AUTOTUNE)


ValueError: Dimensions 5120 and 20 are not compatible

In [22]:
input_shape = (256,256,3)
# Simple Model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)
])


In [23]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')


In [24]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.fit(dataset, epochs=10)
print("Model training complete.")

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_4_1/Cast:0", shape=(None, None, None, 3505, 2480, 3), dtype=float32). Expected shape (None, 256, 256, 3), but input has incompatible shape (None, None, None, 3505, 2480, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, None, 3505, 2480, 3), dtype=uint8)
  • training=True
  • mask=None